In [3]:
import pandas as pd 
import matplotlib.pyplot as plt
import datetime
import neurokit2 as nk
import numpy as np
import hrvanalysis as hrvana
%matplotlib inline

In [4]:
SAMPLING_RATE = 1000
INTERVAL = 5 # We want to take data at every 5 minutes interval

In [5]:
def open_brux_csv(patient_id, week, night_id):
    return pd.read_csv(f"./data/p{patient_id}_w{week}/{night_id}cFnorm.csv")

In [6]:
def get_ECG_data_from_df(df):
    return df["ECG"]

In [7]:
def resample_signal(ecg, sampling_rate=2000, SAMPLING_RATE=SAMPLING_RATE):
    return nk.signal_resample(ecg, sampling_rate=sampling_rate, desired_sampling_rate=SAMPLING_RATE)
    

In [8]:
def get_ecg_array(ecg):
    return ecg.values.tolist()

In [9]:
def get_n_minutes(ecg, n, SAMPLING_RATE=SAMPLING_RATE):
    data_points = n*60*SAMPLING_RATE
    return ecg[:data_points]

In [10]:
def get_signal_duration(ecg, SAMPLING_RATE=SAMPLING_RATE):
    return datetime.timedelta(seconds=(len(ecg)-1)/SAMPLING_RATE)

In [11]:
def find_rr_intervals(cleaned_ecg):
    pantompkins1985 = nk.ecg_findpeaks(cleaned_ecg, method="pantompkins1985") # find the R peaks
    hrv_df = pd.DataFrame(pantompkins1985)
    hrv_df["RR Intervals"] = hrv_df["ECG_R_Peaks"].diff() # calculate the value difference between two adjacent points
    hrv_df.loc[0, "RR Intervals"]=hrv_df.loc[0]['ECG_R_Peaks'] # the first datapoint contain Nan 

    return hrv_df


In [12]:
def clean_rr_intervals(hrv_df):
    clean_rri = hrv_df['RR Intervals'].values
    clean_rri = hrvana.remove_outliers(rr_intervals=clean_rri, low_rri=300, high_rri=2000)
    clean_rri = hrvana.interpolate_nan_values(rr_intervals=clean_rri, interpolation_method="linear")
    clean_rri = hrvana.remove_ectopic_beats(rr_intervals=clean_rri, method="malik")
    clean_rri = hrvana.interpolate_nan_values(rr_intervals=clean_rri, interpolation_method="linear")

    return clean_rri

## Define LF/HF value ranges

### Paper: Herzig et al., Reproducibility of Heart Rate Variability Is Parameter and Sleep Stage Dependent (2017)

In [13]:
# stage_2 = {'min': 0.68, 'median': 1.11, 'max': 2.02}
# sws = {'min': 0.31 , 'median': 0.51, 'max': 0.90}
nrem = {'min': 0.31, 'median': (2.02-0.31)/2 , 'max': 2.02}
rem = {'min': 1.30, 'median': 2.02 , 'max': 3.22}

In [14]:
def get_herzig_ranges():
    return {
        'nrem': {'min': 0.31, 'median': (2.02-0.31)/2 , 'max': 2.02},
        'rem' : {'min': 1.30, 'median': 2.02 , 'max': 3.22}
    }

### Paper: Ako et al., Correlation between electroencephalography and heart rate variability during sleep (2003)

In [15]:
"""
stage_1 = {'min': 2.30-0.29, 'median': 2.30, 'max': 2.30+0.29}
stage_2 = {'min': 1.85-0.09 , 'median': 1.85 , 'max': 1.85+0.09}
stage_3 = {'min': 0.78-0.06 , 'median': 0.78, 'max': 0.78+0.06}
stage_4 = {'min': 0.86-0.14, 'median': 0.86, 'max': 0.86+0.14}
rem = {'min': 2.51-0.17, 'median': 2.51, 'max': 2.51+0.17}
"""

"\nstage_1 = {'min': 2.30-0.29, 'median': 2.30, 'max': 2.30+0.29}\nstage_2 = {'min': 1.85-0.09 , 'median': 1.85 , 'max': 1.85+0.09}\nstage_3 = {'min': 0.78-0.06 , 'median': 0.78, 'max': 0.78+0.06}\nstage_4 = {'min': 0.86-0.14, 'median': 0.86, 'max': 0.86+0.14}\nrem = {'min': 2.51-0.17, 'median': 2.51, 'max': 2.51+0.17}\n"

### Script using Neurokit HRV functions

In [16]:
""""
start = 0
# Number of data points in 5 minutes
window_size = 5*60*SAMPLING_RATE
end = window_size
values = []

while end <= df.index[-1]+1:
    stages = []

    df_5_min = df[start:end]
    ecg = df_5_min["ECG"].values.tolist()
    # Filter the data with ranges specified from Barbara
    filter_ecg = nk.signal_filter(ecg, sampling_rate=SAMPLING_RATE, lowcut=0.5, highcut=150)
    cleaned = nk.ecg_clean(filter_ecg, sampling_rate=SAMPLING_RATE, method="neurokit")
    peaks, info = nk.ecg_peaks(cleaned, sampling_rate=SAMPLING_RATE, method="neurokit")

    hrv_indices = nk.hrv(peaks, sampling_rate=SAMPLING_RATE)
    hrv_welch = nk.hrv_frequency(peaks, sampling_rate=SAMPLING_RATE, psd_method="welch")

    if  nrem['min'] <= hrv_indices['HRV_LFHF'][0] <= nrem['max'] + 1:
        stages.append('nrem')

    elif rem['min'] <= hrv_indices['HRV_LFHF'][0] <= rem['max'] + 1:
        stages.append('rem')

    

    values.append({
        'start_id': start,
        'end_id': end,
        'LF/HF': hrv_welch['HRV_LFHF'][0],
        'SD': hrv_indices['HRV_SDNN'][0],
        'stages': stages,
        'color': None
    })

    

    start = end
    end += window_size
"""

'"\nstart = 0\n# Number of data points in 5 minutes\nwindow_size = 5*60*SAMPLING_RATE\nend = window_size\nvalues = []\n\nwhile end <= df.index[-1]+1:\n    stages = []\n\n    df_5_min = df[start:end]\n    ecg = df_5_min["ECG"].values.tolist()\n    # Filter the data with ranges specified from Barbara\n    filter_ecg = nk.signal_filter(ecg, sampling_rate=SAMPLING_RATE, lowcut=0.5, highcut=150)\n    cleaned = nk.ecg_clean(filter_ecg, sampling_rate=SAMPLING_RATE, method="neurokit")\n    peaks, info = nk.ecg_peaks(cleaned, sampling_rate=SAMPLING_RATE, method="neurokit")\n\n    hrv_indices = nk.hrv(peaks, sampling_rate=SAMPLING_RATE)\n    hrv_welch = nk.hrv_frequency(peaks, sampling_rate=SAMPLING_RATE, psd_method="welch")\n\n    if  nrem[\'min\'] <= hrv_indices[\'HRV_LFHF\'][0] <= nrem[\'max\'] + 1:\n        stages.append(\'nrem\')\n\n    elif rem[\'min\'] <= hrv_indices[\'HRV_LFHF\'][0] <= rem[\'max\'] + 1:\n        stages.append(\'rem\')\n\n    \n\n    values.append({\n        \'start_id\':

## Script using hrvana framework

https://github.com/bzhai/multimodal_sleep_stage_benchmark/blob/master/notebooks/Tutorial-HRV%20Feature%20Extraction%20From%20ECG.ipynb

In [35]:
"""
- patient_id: id of the patient
- week: week number
- night_id: id of the night that specifies which file to open 
- n: first n minutes of data are selected
- SAMPLING_RATE: desired sampling rate (default: 1000)
"""

def get_HRV_features(patient_id, week, night_id, n,  SAMPLING_RATE=SAMPLING_RATE):
    start = 0
    # Number of data points in 5 minutes
    window_size = INTERVAL*60*SAMPLING_RATE
    end = window_size
    values = []


    # Open right CSV file and get ECG data
    df = open_brux_csv(patient_id=patient_id, week=week, night_id=night_id)
    ecg = get_ECG_data_from_df(df)
    ecg_array = get_ecg_array(ecg)

    # Downsample to 1000Hz
    ecg_ds = resample_signal(ecg=ecg_array)

    # Select first n minutes
    ecg_nmin = get_n_minutes(ecg_ds, n, SAMPLING_RATE=SAMPLING_RATE)
    print(f"Signal duration: {get_signal_duration(ecg_nmin)}")
    


    while end <= len(ecg_nmin)+1:
        stages = []

        # 5 minutes of ecg data
        ecg = ecg_nmin[start:end]

        # Filter the data with ranges specified from Barbara
        filter_ecg = nk.signal_filter(ecg, sampling_rate=SAMPLING_RATE, lowcut=0.5, highcut=150)

        # Clean ECG data
        cleaned = nk.ecg_clean(filter_ecg, sampling_rate=SAMPLING_RATE, method="pantompkins1985")

        #Find RR intervals    
        hrv_df = find_rr_intervals(cleaned)

        # Clean RR intervals
    
        clean_rri = clean_rr_intervals(hrv_df)
        hrv_df["RR Intervals"] = clean_rri 

        # HRV feature extraction
        nn_epoch = hrv_df['RR Intervals'].values

        time_features = hrvana.get_time_domain_features(nn_epoch)
        frequency_features = hrvana.get_frequency_domain_features(nn_epoch)

        print(f"LF/HF ratio: {frequency_features['lf_hf_ratio']}")
        nrem = get_herzig_ranges()['nrem']
        rem = get_herzig_ranges()['rem']

        if  nrem['min'] <= frequency_features['lf_hf_ratio'] <= nrem['max']:
            stages.append('nrem')

        elif rem['min'] <= frequency_features['lf_hf_ratio'] <= rem['max']:
            stages.append('rem')

        elif frequency_features['lf_hf_ratio'] > rem['max']:
            stages.append('rem')

        elif frequency_features['lf_hf_ratio'] < nrem['min']:
            stages.append('nrem')

    
        values.append({
            'start_id': start,
            'end_id': end,
            'LF/HF': frequency_features['lf_hf_ratio'],
            'SD': time_features['sdnn'],
            'stages': stages
        })

        

        start = end
        end += window_size
    
    return values

In [36]:
HRV_features = get_HRV_features(1, 1, 1222325, 180)

Signal duration: 2:59:59.999000
25 outlier(s) have been deleted.
The outlier(s) value(s) are : [22951.0, 3417.0, 2761.0, 2838.0, 2183.0, 2374.0, 257.0, 2923.0, 252.0, 5500.0, 4612.0, 287.0, 253.0, 272.0, 2274.0, 258.0, 260.0, 2181.0, 2393.0, 2218.0, 6466.0, 33460.0, 3508.0, 2359.0, 2048.0]
87 ectopic beat(s) have been deleted with malik rule.
LF/HF ratio: 2.0107152456813915
4 outlier(s) have been deleted.
The outlier(s) value(s) are : [2324.0, 266.0, 293.0, 267.0]
31 ectopic beat(s) have been deleted with malik rule.
LF/HF ratio: 1.2472080864024337
3 outlier(s) have been deleted.
The outlier(s) value(s) are : [269.0, 286.0, 257.0]
39 ectopic beat(s) have been deleted with malik rule.
LF/HF ratio: 1.8449693678005492
0 outlier(s) have been deleted.
17 ectopic beat(s) have been deleted with malik rule.
LF/HF ratio: 0.9898579959913515
13 outlier(s) have been deleted.
The outlier(s) value(s) are : [279.0, 251.0, 269.0, 275.0, 260.0, 252.0, 254.0, 6434.0, 257.0, 7473.0, 2513.0, 254.0, 25680.

c:\Users\eleon\Desktop\Master\FJS 23\Master Project\sensor-data-analysis-pipeline\.venv\Lib\site-packages\scipy\signal\_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 118, using nperseg = 118
  warnings.warn('nperseg = {0:d} is greater than input length '
c:\Users\eleon\Desktop\Master\FJS 23\Master Project\sensor-data-analysis-pipeline\.venv\Lib\site-packages\scipy\signal\_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 88, using nperseg = 88
  warnings.warn('nperseg = {0:d} is greater than input length '


89 ectopic beat(s) have been deleted with malik rule.
LF/HF ratio: 1.6673046802209006
133 outlier(s) have been deleted.
The outlier(s) value(s) are : [278.0, 265.0, 255.0, 257.0, 258.0, 277.0, 258.0, 279.0, 281.0, 278.0, 278.0, 258.0, 258.0, 259.0, 257.0, 277.0, 255.0, 277.0, 258.0, 258.0, 256.0, 278.0, 256.0, 256.0, 276.0, 257.0, 255.0, 257.0, 256.0, 256.0, 256.0, 276.0, 257.0, 277.0, 256.0, 277.0, 258.0, 276.0, 276.0, 277.0, 257.0, 256.0, 257.0, 276.0, 265.0, 255.0, 298.0, 270.0, 258.0, 275.0, 294.0, 264.0, 253.0, 258.0, 261.0, 253.0, 258.0, 266.0, 260.0, 253.0, 258.0, 258.0, 261.0, 269.0, 288.0, 257.0, 263.0, 257.0, 259.0, 255.0, 269.0, 256.0, 270.0, 261.0, 274.0, 262.0, 261.0, 264.0, 254.0, 267.0, 264.0, 276.0, 260.0, 254.0, 279.0, 255.0, 256.0, 258.0, 253.0, 265.0, 258.0, 254.0, 267.0, 256.0, 254.0, 273.0, 266.0, 255.0, 286.0, 256.0, 254.0, 261.0, 254.0, 285.0, 259.0, 279.0, 278.0, 254.0, 266.0, 262.0, 279.0, 297.0, 269.0, 259.0, 254.0, 258.0, 269.0, 277.0, 256.0, 276.0, 258.0, 25

In [41]:
y=0
x=0
lst = []
for idx, i in enumerate(HRV_features):
    end = i['end_id']

    minute = round((end-1)/SAMPLING_RATE/60)
    
    if idx != 0:
        if minute == 5:
            y+=1
            x = 0
        
        if minute != 5:
            x+=1

    i['y'] = y
    i['x'] = x

    lst.append(i)





In [42]:
lst

[{'start_id': 0,
  'end_id': 300000,
  'LF/HF': 2.0107152456813915,
  'SD': 290.6453341073007,
  'stages': ['nrem'],
  'y': 0,
  'x': 0},
 {'start_id': 300000,
  'end_id': 600000,
  'LF/HF': 1.2472080864024337,
  'SD': 172.26272979069515,
  'stages': ['nrem'],
  'y': 0,
  'x': 1},
 {'start_id': 600000,
  'end_id': 900000,
  'LF/HF': 1.8449693678005492,
  'SD': 153.53789804010702,
  'stages': ['nrem'],
  'y': 0,
  'x': 2},
 {'start_id': 900000,
  'end_id': 1200000,
  'LF/HF': 0.9898579959913515,
  'SD': 90.94487460120051,
  'stages': ['nrem'],
  'y': 0,
  'x': 3},
 {'start_id': 1200000,
  'end_id': 1500000,
  'LF/HF': 0.7985738057626985,
  'SD': 271.2849888773951,
  'stages': ['nrem'],
  'y': 0,
  'x': 4},
 {'start_id': 1500000,
  'end_id': 1800000,
  'LF/HF': 0.5643732610642004,
  'SD': 122.59924759086121,
  'stages': ['nrem'],
  'y': 0,
  'x': 5},
 {'start_id': 1800000,
  'end_id': 2100000,
  'LF/HF': 0.7853641049298196,
  'SD': 200.37932866639403,
  'stages': ['nrem'],
  'y': 0,
  'x